# Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import joblib
import time
from flair.data import Sentence
from flair.data import Token
from flair.embeddings import WordEmbeddings
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
%matplotlib inline

device = torch.device("cuda:0")
dtype = torch.float

# embedding size of fasttext models
d_model = 300 

# select language
language_in = 'de'
language_out = 'en' 

# DataSet

In [ ]:
class Language_DataSet(torch.utils.data.Dataset):

    def __init__(self, path, language_in, language_out):#D:\Transformer\dataset.data
        super().__init__()
        self.language_in = language_in
        self.language_out = language_out

        # load dat and vocab
        self.data = joblib.load(path+f'{language_in}_to_{language_out}.data')
        self.vocab_in = joblib.load(path+f'vocab_{language_in}.data')
        self.vocab_out = joblib.load(path+f'vocab_{language_out}.data')

        # zero padding info
        self.seq_len_in = self.vocab_in["max_sentence_len"] + 1 # additional <SOS>/<EOS> token
        self.seq_len_out = self.vocab_out["max_sentence_len"] + 1 # additional <SOS>/<EOS> token

        # precompute padding
        self.precompute_padding = torch.zeros(1, d_model).to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # return format: input_encoding, output_encoding, target

        x_in, x_out, target = self.data[index]
        x_in = x_in.to(device)
        x_out = x_out.to(device)
        
        # zero padding

        seq_len_in = x_in.shape[0]
        padding_length_in = self.seq_len_in - seq_len_in

        seq_len_out = x_out.shape[0]
        padding_length_out = self.seq_len_out - seq_len_out

        padding_in = self.precompute_padding.repeat([padding_length_in,1])
        padding_out = self.precompute_padding.repeat([padding_length_out,1])

        x_in = torch.cat([x_in, padding_in], dim=0)
        x_out = torch.cat([x_out, padding_out], dim=0)

        # padding target <EOS> as target for padding?
        target_len = len(target)
        padding_length = self.seq_len_out - target_len
        padding_index = target[-1] # <EOS>

        target = target + [ padding_index for i in range(padding_length)]
        target = torch.tensor(target).to(device)

        return x_in, x_out, target

# Test DataSet

path = "D:\Transformer\\"
dataset = Language_DataSet(path, language_in, language_out)
dl = DataLoader(dataset, batch_size=32,num_workers=4, shuffle=True)
if __name__ == '__main__':
    for eps_data in dl:
        x_in, x_out, target = eps_data
        print(x_in)
        print(x_out)
        print(target)

# Transformer Implementation
## Multi-Head-Attention

In [ ]:
class Multi_Head_Attention(nn.Module):
    
    def __init__(self, heads, seq_len, dimensions):
        super().__init__()
        
        self.heads = heads
        self.dimensions = dimensions
        self.seq_len = seq_len
        
        self.qkv = nn.Linear(d_model, self.dimensions * heads * 3)
        self.final_linear = nn.Linear(self.heads * self.dimensions, d_model)
    
    def forward(self, x, mask=None):
        # x shape: batch,seq_len,d_model
        batch_size, _, _ = x.shape
        
        # reshape for linear qkv layer
        x = torch.reshape(x, (batch_size*self.seq_len, d_model))
        
        # compute q,v,k for every head
        qkv = self.qkv(x)

        # reshape into (batch_size,seq_len,...)
        qkv = torch.reshape(qkv, (batch_size, self.seq_len, self.heads * 3 * self.dimensions))
        # split into heads and seperate q, k, v in different dims
        qkv = torch.reshape(qkv, (batch_size, self.seq_len, self.heads, 3, self.dimensions))
        
        # permute head to front for parallel processing
        qkv = qkv.permute(0,2,1,3,4)
        
        # extract q, k, v
        q = qkv[:,:,:,0,:]
        k = qkv[:,:,:,1,:]
        v = qkv[:,:,:,2,:]
        
        # fuse batch_size and head dim for parallel processing
        q = torch.reshape(q, (batch_size * self.heads, self.seq_len, self.dimensions))
        k = torch.reshape(k, (batch_size * self.heads, self.seq_len, self.dimensions))
        v = torch.reshape(v, (batch_size * self.heads, self.seq_len, self.dimensions))
        
        # transpose k
        k = torch.transpose(k, 1, 2)
        
        # multiply q and k
        qk = torch.bmm(q,k)
        # scale
        qk = qk / torch.sqrt(torch.tensor(self.dimensions).to(device).to(dtype))
        # optional masking
        if mask is not None:
            qk[mask == 1] = float('-inf')
        # softmax
        qk = F.softmax(qk, dim=2)
        
        # multiply with v
        qkv = torch.bmm(qk, v)
        
        # reshape to cat heads
        qkv = torch.reshape(qkv, (batch_size, self.heads, self.seq_len, self.dimensions))
        # cat all heads
        qkv = qkv.permute(0,2,1,3)
        qkv = torch.reshape(qkv, (batch_size, self.seq_len, self.heads * self.dimensions))
        
        # reshape to multiply with final linear
        qkv = torch.reshape(qkv, (batch_size * self.seq_len, self.heads * self.dimensions))
        # multiply with final linear
        z = self.final_linear(qkv)
        
        # reshape to input format
        z = torch.reshape(z, (batch_size, self.seq_len, d_model))
        
        return z

## Encoder Cell

In [ ]:
class Encoder_Cell(nn.Module):
    
    def __init__(self, heads, seq_len, attention_dimension, ff_inner=1024):
        super().__init__()
        self.seq_len = seq_len
        
        self.self_attention = Multi_Head_Attention(heads, seq_len, attention_dimension)
        self.layer_norm_1 = nn.LayerNorm([d_model])
        self.layer_norm_2 = nn.LayerNorm([d_model])
        
        ff_network = [
            nn.Linear(d_model, ff_inner),
            nn.ReLU(),
            nn.Linear(ff_inner, d_model),
        ]
        self.feed_forward_net = nn.Sequential(*ff_network)
        
    def forward(self, x):
        # x shape: batch,seq_len,d_model
        batch_size, _, _ = x.shape
        
        # self attention
        z = self.self_attention(x)
        
        # 1st residual
        residual_1 = x + z
        # 1st norm
        norm_1 = self.layer_norm_1(residual_1)
        
        # reshape norm for feed forward network
        ff_in = torch.reshape(norm_1, (batch_size*self.seq_len, d_model))
        # feed forward
        ff_out = self.feed_forward_net(ff_in)
        # reshape back
        ff_out = torch.reshape(ff_out, (batch_size, self.seq_len, d_model))
        
        # 2nd residual
        residual_2 = norm_1 + ff_out
        # 2nd norm
        norm_2 = self.layer_norm_1(residual_2)
        
        return norm_2

## Encoder

In [ ]:
class Encoder(nn.Module):
    
    def __init__(self, cells, heads, seq_len, attention_dimensions):
        super().__init__()
        self.heads = heads
        self.seq_len = seq_len
        self.attention_dimensions = attention_dimensions
        
        # stacked encoder cells
        encoder_cells = [ Encoder_Cell(heads, seq_len, attention_dimensions).to(device) for i in range(cells)]
        self.encode = nn.Sequential(*encoder_cells)
        
        # key and value output of encoder
        self.kv = nn.Linear(d_model, attention_dimensions * heads * 2)
    
    def forward(self, x):
        
        batch_size = x.shape[0]
        
        # encoding shape: batch_size, seq_len, d_model
        encoding = self.encode(x) 
        
        # reshape to feed into linear kv layer
        encoding = torch.reshape(encoding, (batch_size * self.seq_len, d_model))
        
        # apply linear
        kv = self.kv(encoding)
        # reshape back
        kv = torch.reshape(kv, (batch_size, self.seq_len, self.attention_dimensions * self.heads * 2))
        
        # seperate k and v
        kv = torch.reshape(kv, (batch_size, self.seq_len, self.heads, 2, self.attention_dimensions))
        
        # permute head to front for parallel processing
        kv = kv.permute(0,2,1,3,4)
        
        # split k, v
        k = kv[:,:,:,0,:]
        v = kv[:,:,:,1,:]
        
        # fuse batch_size and head dim for parallel processing
        k = torch.reshape(k, (batch_size * self.heads, self.seq_len, self.attention_dimensions))
        v = torch.reshape(v, (batch_size * self.heads, self.seq_len, self.attention_dimensions))
        
        return k, v

## Encoder-Decoder Attention

In [ ]:
class Encoder_Decoder_Attention(nn.Module):

    def __init__(self, heads, seq_len, attention_dimensions):
        super().__init__()
        self.heads = heads
        self.seq_len = seq_len
        self.attention_dimensions = attention_dimensions
        
        self.q = nn.Linear(d_model, attention_dimensions * heads).to(device)
        self.final_linear = nn.Linear(heads * attention_dimensions, d_model).to(device)
        
    def forward(self, x, encoder_k, encoder_v):
        # x shape: batch_size, seq_len, d_model
        # encoder k/v shape: batch_size*heads, seq_len, attention_dimensions
        batch_size = x.shape[0]
        
        # reshape for linear q layer
        x = torch.reshape(x, (batch_size*self.seq_len, d_model))
        
        # compute q for every head
        q = self.q(x) # (seq * batch, heads * attention_dimensions)

        # reshape into (batch_size,seq_len,...)
        q = torch.reshape(q, (batch_size, self.seq_len, self.heads * self.attention_dimensions))
        # split into heads 
        q = torch.reshape(q, (batch_size, self.seq_len, self.heads, self.attention_dimensions))
        
        # permute head to front for parallel processing
        q = q.permute(0,2,1,3)
        
        # fuse batch_size and head dim for parallel processing
        q = torch.reshape(q, (batch_size * self.heads, self.seq_len, self.attention_dimensions))
        
        # transpose k
        k = torch.transpose(encoder_k, 1, 2)
        
        # multiply q and k
        qk = torch.bmm(q,k)
        # scale
        qk = qk / torch.sqrt(torch.tensor(self.attention_dimensions).to(device).to(dtype))
        # softmax
        qk = F.softmax(qk, dim=2)     
        
        # multiply with v
        qkv = torch.bmm(qk, encoder_v)
        
        # reshape to cat heads
        qkv = torch.reshape(qkv, (batch_size, self.heads, self.seq_len, self.attention_dimensions))
        # cat all heads
        qkv = qkv.permute(0,2,1,3)
        qkv = torch.reshape(qkv, (batch_size, self.seq_len, self.heads * self.attention_dimensions))
        
        # reshape to multiply with final linear
        qkv = torch.reshape(qkv, (batch_size * self.seq_len, self.heads * self.attention_dimensions))
        # multiply with final linear
        z = self.final_linear(qkv)
        
        # reshape to input format
        z = torch.reshape(z, (batch_size, self.seq_len, d_model))
        
        return z     

## Decoder Cell

In [ ]:
class Decoder_Cell(nn.Module):

    def __init__(self, heads, seq_len, attention_dimension, ff_inner=1024):
        super().__init__()
        self.seq_len = seq_len
        self.heads = heads
        
        # construct decoder mask
        a = np.triu(np.ones((seq_len,seq_len)), k=1) 
        self.mask = torch.unsqueeze(torch.tensor(a).to(device).long(),dim=0)
        
        self.self_attention = Multi_Head_Attention(heads, seq_len, attention_dimension).to(device)
        self.enc_dec_attention = Encoder_Decoder_Attention(heads, seq_len, attention_dimension).to(device)
        
        self.layer_norm_1 = nn.LayerNorm([d_model])
        self.layer_norm_2 = nn.LayerNorm([d_model])
        self.layer_norm_3 = nn.LayerNorm([d_model])
        
        ff_network = [
            nn.Linear(d_model, ff_inner),
            nn.ReLU(),
            nn.Linear(ff_inner, d_model),
        ]
        self.feed_forward_net = nn.Sequential(*ff_network)
        
    def forward(self, x, encoder_k, encoder_v):
        # x shape: batch,seq_len,d_model
        batch_size, _, _ = x.shape
        
        # self attention
        mask = self.mask.repeat(batch_size*self.heads,1,1)
        z_1 = self.self_attention(x, mask)
        
        # 1st residual
        residual_1 = x + z_1
        # 1st norm
        norm_1 = self.layer_norm_1(residual_1)
        
        # encoder-decoder attention
        z_2 = self.enc_dec_attention(norm_1, encoder_k, encoder_v)
        
        # 2nd residual
        residual_2 = norm_1 + z_2
        # 2nd norm
        norm_2 = self.layer_norm_2(residual_2)
        
        # reshape norm for feed forward network
        ff_in = torch.reshape(norm_2, (batch_size*self.seq_len, d_model))
        # feed forward
        ff_out = self.feed_forward_net(ff_in)
        # reshape back
        ff_out = torch.reshape(ff_out, (batch_size, self.seq_len, d_model))
        
        # 3rd residual
        residual_3 = norm_2 + ff_out
        # 3rd norm
        norm_3 = self.layer_norm_3(residual_3)
        
        return norm_3

## Decoder

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, cells, heads, seq_len, attention_dimensions, vocab_size):
        super().__init__()
        self.heads = heads
        self.seq_len = seq_len
        self.attention_dimensions = attention_dimensions
        self.vocab_size = vocab_size
        
        # stacked encoder cells
        self.decoder_cells = [ Decoder_Cell(heads, seq_len, attention_dimensions).to(device) for i in range(cells)]
        
        # output layer and then softmax
        self.final_linear = nn.Linear(d_model, vocab_size)
        
    def forward(self, x, encoder_k, encoder_v):
        
        batch_size = x.shape[0]
        
        for decoder_cell in self.decoder_cells:
            x = decoder_cell(x,encoder_k, encoder_v)
            
        # reshape for linear
        x = torch.reshape(x, (batch_size*self.seq_len, d_model))
        
        # feed in final layer
        x = self.final_linear(x)
          
        return x

## Transformer

In [ ]:
class Transformer(nn.Module):
    
    def __init__(self, cells, heads, seq_len_enc, seq_len_dec, attention_dimensions, vocab_size):
        super().__init__()
        
        self.encoder = Encoder(cells, heads, seq_len_enc, attention_dimensions).to(device)
        self.decoder = Decoder(cells, heads, seq_len_dec, attention_dimensions, vocab_size).to(device)
    
    def forward(self, x_encoder, x_decoder):
        # x_in shape: batch_size, seq_len_in, d_model
        # x_out shape: batch_size, seq_len_out, d_model
        
        encoder_k, encoder_v = self.encoder(x_encoder)
        out = self.decoder(x_decoder, encoder_k, encoder_v)
        
        # output needs softmax afterwards | cross_entropy_loss or F.softmax
        return out

## Hyperparameters

In [ ]:
path = "D:\Transformer\\"
vocab_in = joblib.load(path+f'vocab_{language_in}.data')
vocab_out = joblib.load(path+f'vocab_{language_out}.data')

batch_size_dl = 128
epochs = 20
lr = 5e-4
num_cells = 4
cell_embedding_size = 64
num_heads = 6

seq_len_encoder = vocab_in["max_sentence_len"] + 1
seq_len_decoder = vocab_out["max_sentence_len"] + 1
vocab_size = vocab_out["vocab_size"] + 1 # + <EOS>

In [ ]:
transformer = Transformer(num_cells,num_heads,
                          seq_len_encoder,seq_len_decoder,
                          cell_embedding_size,vocab_size).to(device)

print("model weights:",sum(p.numel() for p in transformer.parameters() if p.requires_grad))

dataset = Language_DataSet(path, language_in, language_out)
dataloader = DataLoader(dataset, batch_size=batch_size_dl, num_workers=0, shuffle=True,drop_last=True)

cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer.parameters(), lr=lr)

# Train

In [ ]:
for epoch in range(epochs):
    print("epoch:",epoch)
    
    epoch_start = time.time()
    
    for i, batch in enumerate(dataloader):
        start = time.time()
        optimizer.zero_grad()
        
        # data
        x_in, x_out, target = batch
        
        # reshape target for cross entropy loss
        target = torch.flatten(target)
        
        # run transformer
        out = transformer(x_in, x_out)
        
        loss = cross_entropy_loss(out, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(transformer.parameters(), 0.5)
        optimizer.step()
        
        end = time.time()
        
        if i%25 == 0:
            print(f'i:{i} | loss: {loss} time: {end - start}')
        
    epoch_end = time.time()
    print(f'epoch time: {epoch_end - epoch_start}')

## Save transformer model

torch.save(transformer,f"transformer.pt")

# do inference

## pos encoding

In [ ]:
sine_mask = [2*i for i in range(d_model//2)]
cosine_mask = [2*i+1 for i in range(d_model//2)]
i = torch.tensor([i for i in range(d_model//2)]).to(device).to(dtype)

# pre compute positional encodings for 50 tokens
positional_encodings = []
for position in range(50):
    
    sine = torch.sin( position / ( 10000 ** (2*i/d_model) ) )
    cosine = torch.cos( position / ( 10000 ** (2*i/d_model) ) )
    
    position_enc = torch.zeros(d_model).to(device)
    position_enc[sine_mask] = sine
    position_enc[cosine_mask] = cosine

    positional_encodings.append(position_enc)

# return pre computed pos encoding
def positional_encoding(sentence):
    num_tokens = len(sentence)
    return positional_encodings[0:num_tokens]

In [ ]:
# load word embedding models
embedding_model_en = WordEmbeddings('en')
embedding_model_de = WordEmbeddings('de')

In [ ]:
transformer.eval()
# constructed sentence
sentence_translated = ""

# test sentence
#german = 'Wie alt bist du?'
#german = 'Er ist Rechtsanwalt.'
#german = 'Ich bin sehr begeistert.'
#german = 'Das ist ein Problem.'
#german = 'Ich mag das.'
german = 'Wie viele Menschen gibt es?'

sentence_de = Sentence(german,language_code='de')
# add <EOS> token
sentence_de.add_token("<EOS>")
# embed
embedding_model_de.embed(sentence_de)
# pos encoding
pos_enc = positional_encoding(sentence_de)
# add embedding and pos enc
emb_in = torch.stack([token.embedding + pos_enc[i] for i,token in enumerate(sentence_de)])
# add padding
seq_len_in = emb_in.shape[0]
padding_length_in = vocab_in["max_sentence_len"] + 1 - seq_len_in            
padding = torch.zeros(padding_length_in, d_model).to(device)
                     
x_in = torch.unsqueeze(torch.cat([emb_in, padding], dim=0),dim=0)

# starting point of ouput
sentence_en = Sentence('')
sentence_en.add_token('<SOS>')

done = False
i= 0
while not done and i < vocab_out["max_sentence_len"] + 1:
    #print(sentence_en)
    
    # embed
    embedding_model_en.embed(sentence_en)
    # pos encoding
    pos_enc = positional_encoding(sentence_en)
    # add embedding and pos enc
    emb_out = torch.stack([token.embedding + pos_enc[i] for i,token in enumerate(sentence_en)])
    # add padding
    seq_len_out = emb_out.shape[0]
    padding_length_out = vocab_out["max_sentence_len"] + 1 - seq_len_out            
    padding = torch.zeros(padding_length_out, d_model).to(device)
    x_out = torch.unsqueeze(torch.cat([emb_out, padding], dim=0),dim=0)
    out = transformer(x_in, x_out)
    
    # reshape to batch_size, seq, vocab
    out = torch.reshape(out, (1, seq_len_decoder,vocab_size))
    out = F.softmax(out, dim=2) 
    max_index = torch.argmax(out[0], dim=1).cpu().numpy()
    # get relevant position
    token_index = max_index[i]
    
    token = vocab_out[token_index]
    if token == "<EOS>":
        done = True
    else:
        sentence_translated += token + " "
        print("NEW TOKEN >",token,"<")
        sentence_en.add_token(token)
        
    i += 1
    
print(f'Input: {german} | Translation: {sentence_translated}')